In [1]:
# Datenverarbeitung
import pandas as pd

# Visualisierung
import altair as alt

# Deaktivierung der Warnungen von Altair
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Ermöglicht die Darstellung von umfrangreichen Datensätzen
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
# Globale Variablen
domain = ['precious_metals', 'industrial_metals', 'grains', 'livestocks', 'energy', 'softs']
range_ = ['gold', 'grey', 'green', 'red', 'darkblue', 'lightblue']

In [4]:
df = pd.read_csv("data/commodity_df.csv")

In [5]:
df.head()

,date,year,month,category,mean_category,natural_gas,wti_crude,brent_crude,low_sulphur_gas_oil,uls_diesel,...,corn,soybean_oil,soybean_meal,wheat,hrw_wheat,live_cattle,lean_hogs,sugar,coffee,cotton
0,2000-01-03,2000,1,energy,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-04,2000,1,energy,66.6792,2.176,25.55,24.39,213.50,67.78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-05,2000,1,energy,66.0716,2.168,24.91,23.73,213.00,66.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-06,2000,1,energy,65.6252,2.196,24.78,23.62,211.25,66.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-07,2000,1,energy,63.8966,2.173,24.22,23.09,205.25,64.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Diagramm 3
# Dieses Diagramm ermöglicht einen Vergleich zwischen den Jahren 2007/08 und 2022
# Zusätzlich wurde für die Jahre 2007/08 eine Trennlinie eingefügt, um im Plot den Jahreswechsel zu erkennen

# Plot mit Preisen nach Waren aufgeteilt über den Zeitraum 2007/08
fig_78 = alt.Chart(
    df[(df.year==2007) | (df.year==2008)]
).encode(
    x=alt.X('date:T',
            axis=alt.Axis(title='Monate 2007/2008')),
    y=alt.Y('mean_category:Q',
            axis=alt.Axis(title='Rohstoffpreis (€), Durchschnitt')),
    color=alt.Color('category:N',
                    scale=alt.Scale(domain=domain, range=range_),
                    sort=alt.EncodingSortField('mean_category', op='mean', order='descending'),
                    title='Rohstoffe'
    )
).properties(title='Rohstoffpreise 2007/08, kategorisiert',width=750, height=800)

# Plot mit Preisen nach Waren aufgeteilt über den Zeitraum 2022
fig_22 = alt.Chart(
    df[(df.year==2022)]
).encode(
    x=alt.X('date:T',
            axis=alt.Axis(title='Monate')),
    y=alt.Y('mean_category:Q',
            axis=alt.Axis(title='Rohstoffpreis (€), Durchschnitt')),
    color=alt.Color('category:N',
                    scale=alt.Scale(domain=domain, range=range_),
                    sort=alt.EncodingSortField('mean_category', op='mean', order='descending'),
                    title='Rohstoffe'
    )
).properties(title='Rohstoffpreise 2022, kategorisiert',width=750, height=800)
     
# Graphischer Highlighter der Waren
highlight = alt.selection(type='single', 
                          on='mouseover', 
                          fields=['category'],
                          nearest=True)

points_78 = fig_78.mark_circle(tooltip=True).encode(
    opacity=alt.value(0)
).add_selection(
    highlight
)

lines_78 = fig_78.mark_line(tooltip=True).encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(3.5))
).transform_loess('date', 'mean_category',bandwidth=.005, groupby=['category']).mark_line(size=3)

points_22 = fig_22.mark_circle(tooltip=True).encode(
    opacity=alt.value(0)
).add_selection(
    highlight
)

lines_22 = fig_22.mark_line(tooltip=True).encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(3.5))
)

# Trennlinie der Jahre 2007 und 2008
xrule = (alt.Chart().mark_rule(
    strokeDash=[12, 6], size=1    
).encode(
    x=alt.datum(alt.DateTime(year=2008))
)
)

# Kombination von Plot und Highlighter und Regel
fig_78_highlight = (points_78 + lines_78 + xrule)    
fig_22_highlight = (points_22 + lines_22)

# Verknüpfung der Plots
# die zu vergleichenden Jahreszahlen 2007/08 und 2022 werden horizontal miteinander verknüpft
fig_3 = fig_78_highlight| fig_22_highlight

# Quelle Trennlinie: https://altair-viz.github.io/gallery/line_chart_with_datum.html

In [7]:
# fig_3

Werden die krtischen Jahre direkt verglichen, wird deutlich, dass ihnen unterschiedliche Muster in der Preisentwicklung zugrunde liegen. Für 2022 lässt sich erkennen, dass der Ukraine-Krieg eindeutig auslöser für einen gleichzeitigen Anstieg der Preise von Energie, Getreideprodukten und Industriemetallen ist. Auch Edelmetalle steigen zu dieser Zeit ein wenig an. Im gleichen Maße beruhigen sich die Preissteigerungen aber auch wieder, verbleiben dennoch auch einem höheren Preisniveau, als zu Jahresbeginn noch. <br>
Für die Weltfinanzkrise 2007/08 wird der 09.08.2007 als Starttag genannt. Es fällt auf, dass Industrielle Metalle bereits vorher ihren Höhepunkt in diesem Jahr erreicht haben und eigentlich wieder stark gefallen waren. Ab August, steigen aber auch diese wieder rasant an, begleitet von eineme langsameren, aber ungefähr gleichmäßigen Anstieg von Getreide, Edelmetalle und Energie. Spätestens ein Jahr später, August 2008 beginnt auch die letzte dieser gestiegenen Kategorien wieder zu fallen. <br>
<br>
Der Höhepunkt für industrielle Metalle scheint gleich. Die Preise für Edelmetalle sind über die Jahre bis 2022 stark angestiegen. Getreide ist im Vergleich leicht(er) gestiegen. Vergleicht man die die konkreten beiden Tage des Ausbruchs der jeweiligen Krise, zeigt sich, dass der Ukraine-Krieg deutlich stärkere Auswirkungen auf industrielle Metalle hatte als die Weltfinanzkrise. Das hohe Preisniveau im selben Jahr, begann bereits 2006 und muss demnach auf ein anderes globales Event zurückzuführen sein. Auch war der Anstieg vom Beginn der Krise sehr unterschiedlich. Während die Weltfinanzkrise anscheinend langsamer ihre Auswirkungen auf die Preise entwickelte, war der Beginn des Krieges 2022 wortwörtlich ein Startschuss, ein Schockmoment für die Preisentwicklung.